In [1]:
import pandas as pd

data = pd.read_csv('abcnews-date-text.csv', error_bad_lines=False)

In [2]:
data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [3]:
len(documents)
documents.head()

,headline_text,index
0,aba decides against community broadcasting lic...,0
1,act fire witnesses must be aware of defamation,1
2,a g calls for infrastructure protection summit,2
3,air nz staff in aust strike for pay rise,3
4,air nz strike to affect australian travellers,4


In [18]:
import gensim
import nltk
import numpy as np

from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS

from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *

np.random.seed(2018)
nltk.download('wordnet')

stemmer = SnowballStemmer('english')

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Francisco\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [19]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
            
    return result

In [21]:
doc_sample = documents.iloc[4310][0]

words = doc_sample.split()

print('Original Document: {0}'.format(words))
print('Tokenized and Lemmatized Document: {0}'.format(preprocess(doc_sample)))

Original Document: ['rain', 'helps', 'dampen', 'bushfires']
Tokenized and Lemmatized Document: ['rain', 'help', 'dampen', 'bushfir']


In [28]:
processed_docs = documents['headline_text'].map(preprocess)

processed_docs.head(n=10)

0            [decid, communiti, broadcast, licenc]
1                               [wit, awar, defam]
2           [call, infrastructur, protect, summit]
3                      [staff, aust, strike, rise]
4             [strike, affect, australian, travel]
5               [ambiti, olsson, win, tripl, jump]
6           [antic, delight, record, break, barca]
7    [aussi, qualifi, stosur, wast, memphi, match]
8            [aust, address, secur, council, iraq]
9                         [australia, lock, timet]
Name: headline_text, dtype: object

In [30]:
dictionary = gensim.corpora.Dictionary(processed_docs)

count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 broadcast
1 communiti
2 decid
3 licenc
4 awar
5 defam
6 wit
7 call
8 infrastructur
9 protect
10 summit


In [31]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [34]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]

type(bow_corpus)
bow_corpus[4310]

[(76, 1), (112, 1), (483, 1), (4014, 1)]

In [36]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print('Word {0} (\"{1}\") appears {2} times'.format(
        bow_doc_4310[i],
        dictionary[bow_doc_4310[i][0]],
        bow_doc_4310[i][1]
    ))

Word (76, 1) ("bushfir") appears 1 times
Word (112, 1) ("help") appears 1 times
Word (483, 1) ("rain") appears 1 times
Word (4014, 1) ("dampen") appears 1 times


In [44]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

from pprint import pprint

for doc in corpus_tfidf:
    print(doc)
    pprint(doc)
    break

[(0, 0.5892908867507543), (1, 0.38929654337861147), (2, 0.4964985175717023), (3, 0.5046520327464028)]
[(0, 0.5892908867507543),
 (1, 0.38929654337861147),
 (2, 0.4964985175717023),
 (3, 0.5046520327464028)]


In [46]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=16)

In [48]:
for idx, topic in lda_model.print_topics(num_topics=-1): # todos
    print('Topic: {0} \nWords: {1}')

Topic: 0 
Words: 0.034*"court" + 0.024*"interview" + 0.017*"miss" + 0.015*"polic" + 0.013*"charg" + 0.013*"face" + 0.012*"accus" + 0.012*"search" + 0.010*"arrest" + 0.010*"drug"
Topic: 1 
Words: 0.012*"test" + 0.009*"hous" + 0.009*"say" + 0.008*"south" + 0.008*"research" + 0.008*"rural" + 0.007*"drought" + 0.007*"deal" + 0.006*"elect" + 0.006*"parliament"
Topic: 2 
Words: 0.018*"market" + 0.016*"chang" + 0.012*"warn" + 0.010*"need" + 0.010*"share" + 0.009*"say" + 0.007*"media" + 0.007*"free" + 0.007*"aussi" + 0.007*"busi"
Topic: 3 
Words: 0.013*"final" + 0.009*"interview" + 0.008*"student" + 0.008*"help" + 0.007*"world" + 0.007*"leagu" + 0.006*"australian" + 0.006*"win" + 0.006*"john" + 0.006*"hill"
Topic: 4 
Words: 0.022*"kill" + 0.016*"crash" + 0.015*"death" + 0.015*"attack" + 0.013*"polic" + 0.012*"countri" + 0.012*"dead" + 0.011*"die" + 0.011*"hour" + 0.011*"woman"
Topic: 5 
Words: 0.024*"polic" + 0.017*"charg" + 0.010*"home" + 0.010*"trump" + 0.009*"farm" + 0.008*"guilti" + 0.008*

In [49]:
lda_model_tfidf = gensim.models.LdaMulticore(
    corpus_tfidf, 
    num_topics=10, 
    id2word=dictionary,
    passes=2,
    workers=16
)

for idx, topic in lda_model_tfidf.print_topics(num_topics=-1):
    print('Topic: {0} \nWords: {1}'.format(idx, topic))

Topic: 0 
Words: 0.008*"rural" + 0.007*"trump" + 0.007*"news" + 0.007*"elect" + 0.006*"nation" + 0.005*"climat" + 0.005*"closer" + 0.004*"monday" + 0.004*"tuesday" + 0.004*"chang"
Topic: 1 
Words: 0.006*"australia" + 0.005*"ash" + 0.005*"world" + 0.005*"test" + 0.005*"leagu" + 0.005*"farm" + 0.003*"wind" + 0.003*"april" + 0.003*"plan" + 0.003*"council"
Topic: 2 
Words: 0.009*"market" + 0.005*"share" + 0.005*"blaze" + 0.004*"wednesday" + 0.004*"rate" + 0.004*"novemb" + 0.004*"rise" + 0.004*"hous" + 0.004*"energi" + 0.004*"firefight"
Topic: 3 
Words: 0.011*"interview" + 0.009*"miss" + 0.007*"search" + 0.005*"polic" + 0.004*"michael" + 0.004*"turnbul" + 0.004*"marriag" + 0.003*"homeless" + 0.003*"sydney" + 0.003*"quarter"
Topic: 4 
Words: 0.008*"kill" + 0.006*"crash" + 0.005*"plead" + 0.005*"guilti" + 0.005*"polic" + 0.005*"royal" + 0.004*"dead" + 0.004*"live" + 0.004*"iraq" + 0.004*"blast"
Topic: 5 
Words: 0.015*"charg" + 0.011*"polic" + 0.010*"murder" + 0.010*"court" + 0.007*"assault" +

In [51]:
processed_docs[4310]

['rain', 'help', 'dampen', 'bushfir']

In [52]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1 * tup[1]):
    print('\nScore: {0}\t \nTopic: {1}'.format(score, lda_model.print_topic(index, 10)))


Score: 0.43439173698425293	 
Topic: 0.013*"final" + 0.009*"interview" + 0.008*"student" + 0.008*"help" + 0.007*"world" + 0.007*"leagu" + 0.006*"australian" + 0.006*"win" + 0.006*"john" + 0.006*"hill"

Score: 0.25866708159446716	 
Topic: 0.023*"govern" + 0.014*"china" + 0.012*"green" + 0.011*"australia" + 0.011*"budget" + 0.011*"driver" + 0.009*"plan" + 0.007*"presid" + 0.007*"labor" + 0.007*"polic"

Score: 0.16691485047340393	 
Topic: 0.022*"water" + 0.016*"price" + 0.011*"coast" + 0.010*"gold" + 0.009*"weather" + 0.008*"fall" + 0.007*"fear" + 0.006*"australian" + 0.006*"murray" + 0.006*"hous"

Score: 0.02000589668750763	 
Topic: 0.022*"kill" + 0.016*"crash" + 0.015*"death" + 0.015*"attack" + 0.013*"polic" + 0.012*"countri" + 0.012*"dead" + 0.011*"die" + 0.011*"hour" + 0.011*"woman"

Score: 0.020005496218800545	 
Topic: 0.018*"market" + 0.016*"chang" + 0.012*"warn" + 0.010*"need" + 0.010*"share" + 0.009*"say" + 0.007*"media" + 0.007*"free" + 0.007*"aussi" + 0.007*"busi"

Score: 0.0200

In [53]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.5572489500045776	 
Topic: 0.009*"weather" + 0.007*"water" + 0.005*"flood" + 0.004*"south" + 0.004*"carbon" + 0.004*"north" + 0.004*"west" + 0.004*"farmer" + 0.004*"warn" + 0.003*"rain"

Score: 0.282725065946579	 
Topic: 0.006*"australia" + 0.005*"ash" + 0.005*"world" + 0.005*"test" + 0.005*"leagu" + 0.005*"farm" + 0.003*"wind" + 0.003*"april" + 0.003*"plan" + 0.003*"council"

Score: 0.020007822662591934	 
Topic: 0.009*"market" + 0.005*"share" + 0.005*"blaze" + 0.004*"wednesday" + 0.004*"rate" + 0.004*"novemb" + 0.004*"rise" + 0.004*"hous" + 0.004*"energi" + 0.004*"firefight"

Score: 0.020003702491521835	 
Topic: 0.011*"interview" + 0.009*"miss" + 0.007*"search" + 0.005*"polic" + 0.004*"michael" + 0.004*"turnbul" + 0.004*"marriag" + 0.003*"homeless" + 0.003*"sydney" + 0.003*"quarter"

Score: 0.020003285259008408	 
Topic: 0.015*"charg" + 0.011*"polic" + 0.010*"murder" + 0.010*"court" + 0.007*"assault" + 0.007*"crash" + 0.007*"driver" + 0.007*"accus" + 0.007*"woman" + 0.006*"dru

In [54]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

In [55]:
for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.31976088881492615	 Topic: 0.018*"market" + 0.016*"chang" + 0.012*"warn" + 0.010*"need" + 0.010*"share"
Score: 0.20543114840984344	 Topic: 0.024*"polic" + 0.017*"charg" + 0.010*"home" + 0.010*"trump" + 0.009*"farm"
Score: 0.19117413461208344	 Topic: 0.012*"test" + 0.009*"hous" + 0.009*"say" + 0.008*"south" + 0.008*"research"
Score: 0.18361763656139374	 Topic: 0.022*"kill" + 0.016*"crash" + 0.015*"death" + 0.015*"attack" + 0.013*"polic"
Score: 0.016672099009156227	 Topic: 0.023*"govern" + 0.014*"china" + 0.012*"green" + 0.011*"australia" + 0.011*"budget"
Score: 0.016670363023877144	 Topic: 0.019*"open" + 0.018*"health" + 0.014*"nation" + 0.012*"servic" + 0.011*"push"
Score: 0.016668878495693207	 Topic: 0.025*"council" + 0.016*"melbourn" + 0.011*"plan" + 0.009*"work" + 0.008*"power"
Score: 0.01666880212724209	 Topic: 0.034*"court" + 0.024*"interview" + 0.017*"miss" + 0.015*"polic" + 0.013*"charg"
Score: 0.01666834205389023	 Topic: 0.022*"water" + 0.016*"price" + 0.011*"coast" + 0